Step 1 : Load Dataset Load the dataset after applying SMOTE, ensuring the dataset is balanced.

In [70]:
import pandas as pd

# Load your balanced dataset (after SMOTE application)
df = pd.read_csv("balanced_dataset.csv")


Label                     1.000000
id                        0.987731
URL_Depth                 0.960224
Has_HTTPS                 0.925687
Num_Subdomains            0.902427
Num_Path_Segments         0.811359
Has_IP                    0.778206
URL_Length                0.729077
digit_letter_ratio        0.698226
https_www_combo           0.135477
Have_QuestionMark         0.057398
Have_Equals               0.055865
Query_Parameter_Count     0.048750
Have_Percent              0.047806
has_encoded_chars         0.047806
Has_Suspicious_Keyword    0.009546
Have_At                   0.003391
Has_URL_Shortener        -0.086451
Has_Port                       NaN
Name: Label, dtype: float64


In [183]:
df.head()

,id,dateadded,url,url_status,last_online,threat,tags,urlhaus_link,reporter,Label,...,Has_Suspicious_Keyword,Has_IP,Query_Parameter_Count,Has_Port,Num_Path_Segments,Has_HTTPS,Has_URL_Shortener,digit_letter_ratio,has_encoded_chars,https_www_combo
0,3230050,2024-10-11 12:23:33,http://117.209.4.108:57478/bin.sh,offline,2024-10-11 12:23:33,malware_download,"32-bit,elf,mips,Mozi",https://urlhaus.abuse.ch/url/3230050/,geenensp,1,...,0,1,0,0,1,0,0,1.666666,0,0
1,3177228,2024-09-16 19:43:06,http://154.216.18.121/.c/loginclientbot,offline,2024-09-20 07:22:27,malware_download,"elf,ladvix,opendir",https://urlhaus.abuse.ch/url/3177228/,NDA0E,1,...,1,1,0,0,2,0,0,0.578947,0,0
2,815,2024-11-30 02:10:35,zoho.com,online,2024-11-30 02:10:35,legitimate,none,NaN,Tranco-58PNN,0,...,0,0,0,0,0,-1,0,0.000000,0,0
3,792779,2024-11-30 02:10:35,kp-velenje.si,online,2024-11-30 02:10:35,legitimate,none,NaN,Tranco-58PNN,0,...,0,0,0,0,0,-1,0,0.000000,0,0
4,3157557,2024-09-05 06:43:05,http://119.117.99.166:41825/i,offline,2024-09-10 05:51:25,malware_download,"32-bit,elf,mips,Mozi",https://urlhaus.abuse.ch/url/3157557/,geenensp,1,...,0,1,0,0,1,0,0,3.199999,0,0


Handling Missing Data

In [185]:
# Check the initial state of the DataFrame
print("Initial DataFrame:")
print(df)

# Identify columns with null (NaN) values
columns_with_nan = df.columns[df.isnull().any()]
print("\nColumns with NaN values:")
print(columns_with_nan)

# Drop columns containing NaN values
df_cleaned = df.drop(columns=columns_with_nan)

# Display the cleaned DataFrame
print("\nDataFrame after removing columns with NaN values:")
print(df_cleaned)

Initial DataFrame:
             id            dateadded                                      url  \
0       3230050  2024-10-11 12:23:33        http://117.209.4.108:57478/bin.sh   
1       3177228  2024-09-16 19:43:06  http://154.216.18.121/.c/loginclientbot   
2           815  2024-11-30 02:10:35                                 zoho.com   
3        792779  2024-11-30 02:10:35                            kp-velenje.si   
4       3157557  2024-09-05 06:43:05            http://119.117.99.166:41825/i   
...         ...                  ...                                      ...   
347769    49718  2024-11-30 02:10:35                           clarovideo.net   
347770  3300740  2024-11-23 10:11:12    http://linux-it.abuser.eu/yakuza.m68k   
347771   644609  2024-11-30 02:10:35                             pgtmdssy.com   
347772   168334  2024-11-30 02:10:35                       elfinancierocr.com   
347773   224082  2024-11-30 02:10:35                               godrej.com   

       u

In [8]:
# Drop the unnecessary columns 'dateadded' and 'reporter'
df_cleaned = df.drop(columns=['dateadded', 'last_online','reporter','tags','urlhaus_link'])


In [10]:
print(df_cleaned)

             id                                      url url_status  \
0       3230050        http://117.209.4.108:57478/bin.sh    offline   
1       3177228  http://154.216.18.121/.c/loginclientbot    offline   
2           815                                 zoho.com     online   
3        792779                            kp-velenje.si     online   
4       3157557            http://119.117.99.166:41825/i    offline   
...         ...                                      ...        ...   
347769    49718                           clarovideo.net     online   
347770  3300740    http://linux-it.abuser.eu/yakuza.m68k    offline   
347771   644609                             pgtmdssy.com     online   
347772   168334                       elfinancierocr.com     online   
347773   224082                               godrej.com     online   

                  threat  Label  URL_Length  URL_Depth  Have_At  Have_Percent  \
0       malware_download      1          33          3        0   

Feature Encoding

GRU models process numerical data, so categorical variables need to be encoded.

Label Encoding for categorical variables (like url_status and threat):

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_cleaned['url_status'] = le.fit_transform(df_cleaned['url_status'])
df_cleaned['threat'] = le.fit_transform(df_cleaned['threat'])


In [15]:
from sklearn.preprocessing import LabelEncoder

# Columns to encode
categorical_columns = ['url_status', 'threat']

# Initialize LabelEncoder
label_encoders = {}

# Encode each categorical column
for column in categorical_columns:
    le = LabelEncoder()
    df_cleaned[column] = le.fit_transform(df_cleaned[column])
    label_encoders[column] = le  # Store the encoder for inverse transform if needed
# Convert all columns except 'url' to integers
columns_to_convert = df_cleaned.columns.difference(['url'])  # Exclude 'url' column
df_cleaned[columns_to_convert] = df_cleaned[columns_to_convert].astype(int)

# Verify the conversion
print(df_cleaned.dtypes)

# Check the encoded DataFrame
print("DataFrame after Label Encoding:")
print(df_cleaned.head())

# Optional: Check the mapping of encoded values
for column, le in label_encoders.items():
    print(f"\nMapping for {column}:")
    print(dict(zip(le.classes_, le.transform(le.classes_))))

id                         int64
url                       object
url_status                 int64
threat                     int64
Label                      int64
URL_Length                 int64
URL_Depth                  int64
Have_At                    int64
Have_Percent               int64
Have_Equals                int64
Have_QuestionMark          int64
Num_Subdomains             int64
Has_Suspicious_Keyword     int64
Has_IP                     int64
Query_Parameter_Count      int64
Has_Port                   int64
Num_Path_Segments          int64
Has_HTTPS                  int64
Has_URL_Shortener          int64
digit_letter_ratio         int64
has_encoded_chars          int64
https_www_combo            int64
dtype: object
DataFrame after Label Encoding:
        id                                      url  url_status  threat  \
0  3230050        http://117.209.4.108:57478/bin.sh           0       1   
1  3177228  http://154.216.18.121/.c/loginclientbot           0       1   
2  

Tokenize URLs:

Break URLs into sequences of characters or words for GRU input. Use tools like Tokenizer from keras.preprocessing.text

In [19]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install Keras

Note: you may need to restart the kernel to use updated packages.


Tokenize URLs: Break URLs into sequences of characters or words for GRU input. Use tools like Tokenizer from keras.preprocessing.text.

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Assuming df_cleaned is your DataFrame with 'url' and 'Label' columns

# Step 1: Tokenize URLs at the character level
tokenizer = Tokenizer(char_level=True)  # Tokenizing at the character level (i.e., each character is a token)
tokenizer.fit_on_texts(df_cleaned['url'])  # Fit the tokenizer on URLs

# Convert URLs to sequences of integers (where each integer represents a character)
X = tokenizer.texts_to_sequences(df_cleaned['url'])

# Step 2: Pad sequences to ensure uniform length across all URLs
X = pad_sequences(X, padding='post')  # Pad sequences so they all have the same length

# Step 3: Get the target variable (Label)
y = df_cleaned['Label']  # Assuming 'Label' column contains the phishing label (0 or 1)


In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize and pad the URLs
tokenizer = Tokenizer(num_words=5000)  # Limit to top 5000 unique tokens
tokenizer.fit_on_texts(df['url'])  # Fit on the entire dataset

# Convert URLs to sequences
X_seq = tokenizer.texts_to_sequences(df['url'])

# Pad sequences to a fixed length (e.g., 100)
X_seq_padded = pad_sequences(X_seq, maxlen=100)

In [125]:
# Drop highly correlated features
df_cleaned = df.drop(['id', 'URL_Depth', 'Has_HTTPS', 'Num_Subdomains'], axis=1)
print(df_cleaned.head())
X_seq_train = pd.DataFrame(X_seq_train).reset_index(drop=True)
X_train = pd.DataFrame(X_train).reset_index(drop=True)
y_train = pd.Series(y_train).reset_index(drop=True)

X_seq_val = pd.DataFrame(X_seq_val).reset_index(drop=True)
X_val = pd.DataFrame(X_val).reset_index(drop=True)
y_val = pd.Series(y_val).reset_index(drop=True)


             dateadded                                      url url_status  \
0  2024-10-11 12:23:33        http://117.209.4.108:57478/bin.sh    offline   
1  2024-09-16 19:43:06  http://154.216.18.121/.c/loginclientbot    offline   
2  2024-11-30 02:10:35                                 zoho.com     online   
3  2024-11-30 02:10:35                            kp-velenje.si     online   
4  2024-09-05 06:43:05            http://119.117.99.166:41825/i    offline   

           last_online            threat                  tags  \
0  2024-10-11 12:23:33  malware_download  32-bit,elf,mips,Mozi   
1  2024-09-20 07:22:27  malware_download    elf,ladvix,opendir   
2  2024-11-30 02:10:35        legitimate                  none   
3  2024-11-30 02:10:35        legitimate                  none   
4  2024-09-10 05:51:25  malware_download  32-bit,elf,mips,Mozi   

                            urlhaus_link      reporter  Label  URL_Length  \
0  https://urlhaus.abuse.ch/url/3230050/      geenensp   

Split Data for Training and Testing and Validation

In [170]:
from sklearn.model_selection import train_test_split
import numpy as np

# Shuffle the dataset to ensure randomness
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract features and target
X_features = df_shuffled[selected_features].to_numpy()  # Manually engineered features
y = df_shuffled['Label'].to_numpy()  # Target variable
ids = df_shuffled['id'].to_numpy()  # Unique identifiers

# Combine sequential and feature inputs
X_combined = np.hstack((X_seq_padded, X_features))

# First split: Training (70%) and Temp (30% for validation + testing)
X_train, X_temp, y_train, y_temp, ids_train, ids_temp = train_test_split(
    X_combined, y, ids, test_size=0.3, random_state=42, shuffle=False
)

# Second split: Validation (20%) and Testing (10%) from Temp
X_val, X_test, y_val, y_test, ids_val, ids_test = train_test_split(
    X_temp, y_temp, ids_temp, test_size=0.33, random_state=42, shuffle=False
)

# Separate sequential and feature inputs
seq_length = X_seq_padded.shape[1]
X_seq_train, X_train = X_train[:, :seq_length], X_train[:, seq_length:]
X_seq_val, X_val = X_val[:, :seq_length], X_val[:, seq_length:]
X_seq_test, X_test = X_test[:, :seq_length], X_test[:, seq_length:]


# Remove overlapping IDs from the validation set
ids_val_clean = np.array([id_ for id_ in ids_val if id_ not in train_ids])
X_val_clean = np.array([row for i, row in enumerate(X_val) if ids_val[i] not in train_ids])
y_val_clean = np.array([label for i, label in enumerate(y_val) if ids_val[i] not in train_ids])
# Recheck overlaps
train_ids = set(ids_train)  # IDs in the training set
val_ids_clean = set(ids_val_clean)  # IDs in the cleaned validation set

common_ids_train_val = train_ids.intersection(val_ids_clean)
print(f"Number of overlapping IDs between training and cleaned validation: {len(common_ids_train_val)}")
print(f"Training set size: {X_train.shape}, Labels: {y_train.shape}")
print(f"Validation set size: {X_val.shape}, Labels: {y_val.shape}")
print(f"Testing set size: {X_test.shape}, Labels: {y_test.shape}")


Number of overlapping IDs between training and cleaned validation: 0
Training set size: (243441, 9), Labels: (243441,)
Validation set size: (69903, 9), Labels: (69903,)
Testing set size: (34430, 9), Labels: (34430,)


Model Development

In [174]:
# Import libraries for deep learning
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


Define the GRU Model Architecture Here, we'll build a simple GRU-based neural network to classify phishing URLs. We'll add an embedding layer to represent the tokenized URLs as dense vectors and a GRU layer to learn dependencies from sequential data.

In [176]:

# Sequential input for tokenized URLs
seq_input = Input(shape=(100,), name='seq_input')  # Assuming 100 is the max sequence length
embedding = Embedding(input_dim=5000, output_dim=16, name='embedding_layer')(seq_input)  # 5000 vocab size, 16-dim embeddings
gru_output = GRU(8, return_sequences=False, name='gru_layer')(embedding)  # Reduced GRU units to 8
gru_dropout = Dropout(0.5, name='gru_dropout')(gru_output)  # Dropout rate of 50%

# Auxiliary input for manually engineered features
feat_input = Input(shape=(len(selected_features),), name='feat_input')  # Length of selected_features
feat_dense = Dense(16, activation='relu', name='feat_dense')(feat_input)  # Dense layer for feature input
feat_dropout = Dropout(0.5, name='feat_dropout')(feat_dense)  # Dropout rate of 50%

# Combine GRU and auxiliary features
combined = Concatenate(name='concatenate_layer')([gru_dropout, feat_dropout])
combined_dense = Dense(16, activation='relu', name='combined_dense')(combined)  # Dense layer after combining
final_output = Dense(1, activation='sigmoid', name='output_layer')(combined_dense)  # Sigmoid output for binary classification

# Compile the model
model = Model(inputs=[seq_input, feat_input], outputs=final_output)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ seq_input           │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_layer     │ (None, 100, 16)   │     80,000 │ seq_input[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feat_input          │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_layer (GRU)     │ (None, 8)         │        624 │ embedding_layer[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feat_dense (Dense)  │ (None, 16)        │        160 │ feat_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_dropout         │ (None, 8)         │          0 │ gru_layer[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feat_dropout        │ (None, 16)        │          0 │ feat_dense[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_layer   │ (None, 24)        │          0 │ gru_dropout[0][0… │
│ (Concatenate)       │                   │            │ feat_dropout[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ combined_dense      │ (None, 16)        │        400 │ concatenate_laye… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 1)         │         17 │ combined_dense[0… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 81,201 (317.19 KB)

 Trainable params: 81,201 (317.19 KB)

 Non-trainable params: 0 (0.00 B)

Model Training and Hyperparameter Tuning

Train the Model
Train the GRU model with a validation split for monitoring overfitting.


In [181]:
history = model.fit(
    [X_seq_train, X_train],  # Sequential and feature inputs
    y_train,                 # Target labels
    validation_data=([X_seq_val, X_val], y_val),  # Validation data
    epochs=5,               # Train for 10 epochs
    batch_size=64,           # Batch size of 64
    verbose=1                # Display training progress
)


Epoch 1/5
3804/3804 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - accuracy: 0.9829 - loss: 0.0483 - val_accuracy: 0.9494 - val_loss: 0.1474
Epoch 2/5
3804/3804 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - accuracy: 0.9833 - loss: 0.0456 - val_accuracy: 0.9469 - val_loss: 0.1613
Epoch 3/5
3804/3804 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - accuracy: 0.9834 - loss: 0.0439 - val_accuracy: 0.9463 - val_loss: 0.1644
Epoch 4/5
3804/3804 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - accuracy: 0.9836 - loss: 0.0438 - val_accuracy: 0.9501 - val_loss: 0.1505
Epoch 5/5
3804/3804 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - accuracy: 0.9830 - loss: 0.0436 - val_accuracy: 0.9540 - val_loss: 0.1341


Hyperparameter Tuning
Use Keras Tuner to find the best hyperparameters for the GRU model.

In [ ]:
import keras_tuner as kt

# Define model for tuning
def build_model(hp):
    model = Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train.shape[1], 1)))

    # Add tunable GRU layers
    for i in range(hp.Int('num_layers', 1, 3)):  # Number of GRU layers
        model.add(GRU(
            units=hp.Int(f'gru_units_{i}', min_value=32, max_value=128, step=32),
            return_sequences=True if i < hp.Int('num_layers', 1, 3) - 1 else False
        ))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Create the tuner
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory='tuning_dir',
    project_name='gru_hyperparameter_tuning'
)

# Perform the search
tuner.search(X_train_reshaped, y_train, epochs=10, validation_data=(X_val_reshaped, y_val))

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Learning Rate: {best_hps.get('learning_rate')}")
print(f"Number of GRU Layers: {best_hps.get('num_layers')}")
for i in range(best_hps.get('num_layers')):
    print(f"GRU Units in Layer {i}: {best_hps.get(f'gru_units_{i}')}")
    print(f"Dropout Rate in Layer {i}: {best_hps.get(f'dropout_{i}')}")


train the Best Model
Train the model with the best hyperparameters found during tuning.

In [ ]:
# Build the best model
best_model = tuner.hypermodel.build(best_hps)

# Train the best model
history = best_model.fit(
    X_train_reshaped, y_train,
    validation_data=(X_val_reshaped, y_val),
    epochs=20,
    batch_size=32
)


Evaluate the Model
Evaluate the model on the test set using standard metrics

In [ ]:
# Reshape test data
X_test_reshaped = X_test.reshape(-1, X_test.shape[1], 1)

# Evaluate the model
test_loss, test_accuracy = best_model.evaluate(X_test_reshaped, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predict on test data
y_pred = (best_model.predict(X_test_reshaped) > 0.5).astype(int)

# Calculate evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

Visualize Training Progress
Plot accuracy and loss for training and validation.

In [ ]:
# Plot accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()